In [28]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [29]:
#Import Dependencies 
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [30]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [31]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [32]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Prepare ETL for the Course data

In [33]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [34]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [35]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [36]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

# Deleting  courses records  

In [37]:
course_records = sf.query("SELECT Name FROM Course__c")
recs_to_delete = [{'Name': r['Name']} for r in course_records['records']]
recs_to_delete

[]

In [38]:
for rec in recs_to_delete:
    try:
        sf.Course__c.delete(rec['Name'])
    except Exception as e:
        print(e)

# INSERT DATA INTO COURSE TABLE

In [40]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Bootcamp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

# Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the Course table

It is important to note that we will be querying Salesforce to retrieve the record IDs

In [41]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, Course_Code__c FROM Course__c")
for row in data:
    rec = {
        'Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df.head()

,Course__c,Course_Code__c
0,a014x0000096Q2n,CIS-435
1,a014x0000096Q2d,CIS-438
2,a014x0000096Q2J,BC-WEBDEV
3,a014x0000096Q2i,CIS-430
4,a014x0000096Q2T,CIS-349


In [42]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cl.StartDate AS Start_Date__c,
        cl.EndDate AS End_Date__c       
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


In [43]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14)},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8)},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)}]

In [44]:
class_xwalk_df = pd.merge(class_data_df, course_lookup_df, how='left')
class_xwalk_df.head()

class_xwalk_recs = class_xwalk_df.to_dict(orient='records')
class_xwalk_recs[0]

{'Course_Code__c': 'BC-DATAVIZ',
 'Section__c': 'GWU-ARL-DATA-PT-09-0',
 'Start_Date__c': datetime.date(2020, 9, 16),
 'End_Date__c': datetime.date(2020, 3, 14),
 'Course__c': 'a014x0000096Q2E'}

# Deleting class records

In [45]:
class_records = sf.query("SELECT Name FROM Class__c")
recs_to_delete = [{'Name': r['Name']} for r in class_records['records']]
recs_to_delete

[]

In [46]:
for rec in recs_to_delete:
    try:
        sf.class__c.delete(rec['Name'])
    except Exception as e:
        print(e)

In [49]:
for rec in class_xwalk_recs:

    record = {
        'ID_Course__c': rec['Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'EndDate__c': str(rec['End_Date__c']),
    }
    print(record)
    
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)

{'ID_Course__c': 'a014x0000096Q2E', 'Section__c': 'GWU-ARL-DATA-PT-09-0', 'Start_Date__c': '2020-09-16', 'EndDate__c': '2020-03-14'}
{'ID_Course__c': 'a014x0000096Q2E', 'Section__c': 'GWDC201805DATA3', 'Start_Date__c': '2018-05-15', 'EndDate__c': '2018-11-08'}
{'ID_Course__c': 'a014x0000096Q2J', 'Section__c': 'GWARL201905WEB3', 'Start_Date__c': '2019-05-14', 'EndDate__c': '2019-11-07'}
{'ID_Course__c': 'a014x0000096Q2O', 'Section__c': 'GWARL201905UIUX3', 'Start_Date__c': '2019-05-14', 'EndDate__c': '2019-11-07'}


# Prepare ETL for the student data

In [50]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [51]:
student_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'Birthdate__c',
    'Gender':'Gender__c',
}, inplace=True)
student_data_df.head()

,ID_Student,StudentID__c,LastName__c,FirstName__c,MiddleName__c,Birthdate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [52]:
student_data_df = student_data_df[['StudentID__c', 'LastName__c', 'FirstName__c', 'MiddleName__c', 'Birthdate__c', 'Gender__c']]
student_data_df.head()

,StudentID__c,LastName__c,FirstName__c,MiddleName__c,Birthdate__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [53]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Precious',
  'MiddleName__c': 'Mariette',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '

# Deleting student record

In [54]:
student_records = sf.query("SELECT Name FROM Student__c")
recs_to_delete = [{'Name': r['Name']} for r in student_records['records']]
recs_to_delete

[]

In [55]:
for rec in recs_to_delete:
    try:
        sf.Student__c.delete(rec['Name'])
    except Exception as e:
        print(e)

In [56]:
len(student_data_records)

30

In [58]:
for rec in student_data_records:

    record = {
        'StudentID__c': rec['StudentID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'Birthdate__c': rec['Birthdate__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)
        

# Prepare ETL for the staff data

In [59]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [60]:
staff_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'Birthdate__c',
   
}, inplace=True)
staff_data_df.head()

,ID_Staff,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,Birthdate__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [61]:
staff_data_df = staff_data_df[['EmployeeID__c', 'LastName__c', 'FirstName__c', 'MiddleName__c', 'Birthdate__c']]
staff_data_df.head()

,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,Birthdate__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [62]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'EmployeeID__c': '000184220',
  'LastName__c': 'Luongo',
  'FirstName__c': 'Darick',
  'MiddleName__c': 'Nico',
  'Birthdate__c': datetime.date(1995, 7, 5)},
 {'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'Birthdate__c': datetime.date(1992, 4, 22)},
 {'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'Birthdate__c': datetime.date(1993, 5, 21)},
 {'EmployeeID__c': '000159108',
  'LastName__c': 'Rodney',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Nicole',
  'Birthdate__c': datetime.date(1994, 6, 7)}]

# Deleting staff record

In [63]:
staff_records = sf.query("SELECT Name FROM staff__c")
recs_to_delete = [{'Name': r['Name']} for r in staff_records['records']]
recs_to_delete

[]

In [64]:
for rec in recs_to_delete:
    try:
        sf.Staff__c.delete(rec['Name'])
    except Exception as e:
        print(e)

# INSERTING DATA INTO STAFF TABLE

In [66]:
for rec in staff_data_records:

    record = {
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'Birthdate__c': str(rec['Birthdate__c']),
        
    }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)
        

# Query Student Records from MySQL¶


# Create student lookup Table

In [67]:
query = '''
    SELECT 
        StudentID AS StudentID__c,
        LastName AS LastName__c,
        FirstName AS FirstName__c,
        MiddleName AS MiddleName__c,
        BirthDate AS BirthDate__c,
        Gender AS Gender__c
    FROM
        student
'''

student_df = pd.read_sql(query, conn)

print(student_df.head())

student_dict = student_df.to_dict(orient='records')

student_dict

  StudentID__c LastName__c FirstName__c MiddleName__c BirthDate__c Gender__c
0     25004961    Dartling      Heather         Alice         None         F
1     25003514    Dartling         Lana       Cecille         None         F
2     25005833    Dartling      Jessica       Dorothy         None         F
3     25002589    Dartling     Kimberly     Genevieve         None         F
4     25007185    Dartling    Katherine      Lynnette         None         F


[{'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Precious',
  'MiddleName__c': 'Mariette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '

In [68]:
try:
    for rec in student_dict:
        sf.Student__c.create(rec)
except Exception as e:
    print(e)

# DELETE RECORDS FOR CLASSPARTICIPANT

In [71]:
class_participant_records_del = sf.query("SELECT Name FROM ClassParticipant__c")
recs_to_delete = [{'Name': r['Name']} for r in class_participant_records_del['records']]
recs_to_delete

for rec in recs_to_delete:
    try:
        sf.Class_Participant__c.delete(rec['Name'])
    except Exception as e:
        print(e)



# Load Class Participant Example

In [72]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a014x0000096Q2n,CIS-435
1,a014x0000096Q2d,CIS-438
2,a014x0000096Q2J,BC-WEBDEV
3,a014x0000096Q2i,CIS-430
4,a014x0000096Q2T,CIS-349


In [73]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a014x0000096Q2n,CIS-435
1,a014x0000096Q2d,CIS-438
2,a014x0000096Q2J,BC-WEBDEV
3,a014x0000096Q2i,CIS-430
4,a014x0000096Q2T,CIS-349


In [74]:
sf_class_records = []

sf_class_data = sf.query('SELECT Name, ID_Course__c, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Class__c': row['Name'],
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
 
sf_class_records

[{'ID_Class__c': 'a004x0000036LW4',
  'ID_Course__c': 'a014x0000096Q2OAAU',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a004x0000036LVp',
  'ID_Course__c': 'a014x0000096Q2EAAU',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a004x0000036LVu',
  'ID_Course__c': 'a014x0000096Q2EAAU',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a004x0000036LVz',
  'ID_Course__c': 'a014x0000096Q2JAAU',
  'Section__c': 'GWARL201905WEB3'}]

In [75]:
for rec in sf_class_records:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]

In [76]:
sf_class_df = pd.DataFrame(sf_class_records)

#sf_class_df['ID_Course__c'] = sf_class_df['ID_Course__c'].apply(lambda id_course: id_course[:-3])

sf_class_df.head()

,ID_Class__c,ID_Course__c,Section__c
0,a004x0000036LW4,a014x0000096Q2O,GWARL201905UIUX3
1,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0
2,a004x0000036LVu,a014x0000096Q2E,GWDC201805DATA3
3,a004x0000036LVz,a014x0000096Q2J,GWARL201905WEB3


In [77]:
sf_class_df

,ID_Class__c,ID_Course__c,Section__c
0,a004x0000036LW4,a014x0000096Q2O,GWARL201905UIUX3
1,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0
2,a004x0000036LVu,a014x0000096Q2E,GWDC201805DATA3
3,a004x0000036LVz,a014x0000096Q2J,GWARL201905WEB3


In [78]:
sf_course_df

,ID_Course__c,Course_Code__c
0,a014x0000096Q2n,CIS-435
1,a014x0000096Q2d,CIS-438
2,a014x0000096Q2J,BC-WEBDEV
3,a014x0000096Q2i,CIS-430
4,a014x0000096Q2T,CIS-349
5,a014x0000096Q2E,BC-DATAVIZ
6,a014x0000096Q2O,BC-UIUX
7,a014x0000096Q2Y,CIS-405


In [79]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df, on='ID_Course__c')
sf_class_xwalk


#a044x000002rs51AAA
#a044x000002rs51

sf_class_xwalk

,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,a004x0000036LW4,a014x0000096Q2O,GWARL201905UIUX3,BC-UIUX
1,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,a004x0000036LVu,a014x0000096Q2E,GWDC201805DATA3,BC-DATAVIZ
3,a004x0000036LVz,a014x0000096Q2J,GWARL201905WEB3,BC-WEBDEV


In [80]:
sf_class_df

,ID_Class__c,ID_Course__c,Section__c
0,a004x0000036LW4,a014x0000096Q2O,GWARL201905UIUX3
1,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0
2,a004x0000036LVu,a014x0000096Q2E,GWDC201805DATA3
3,a004x0000036LVz,a014x0000096Q2J,GWARL201905WEB3


In [81]:
sf_student_records = []

sf_student_data = sf.query('SELECT Name, StudentID__c FROM Student__c')

for row in sf_student_data['records']:
    
    rec = {
        'ID_Student__c': row['Name'],
        'StudentID__c': row['StudentID__c']
    }
    
    sf_student_records.append(rec)
    
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,StudentID__c
0,a024x000002cx1O,25003773
1,a024x000002cwzh,25004961
2,a024x000002cwzr,25005833
3,a024x000002cx0G,25003778
4,a024x000002cx0V,25004152


In [82]:
query = '''

    SELECT
        s.StudentID AS StudentID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS StartDate__c,
        cp.EndDate AS EndDate__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

classparticiant_df = pd.read_sql(query, conn)
classparticiant_df.head()

,StudentID__c,Course_Code__c,Section__c,StartDate__c,EndDate__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [83]:
#sf_class_xwalk
#sf_student_xwalk

class_participant_load = pd.merge(classparticiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])
class_participant_load.head()

,StudentID__c,Course_Code__c,Section__c,StartDate__c,EndDate__c,ID_Class__c,ID_Course__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E


In [84]:
sf_class_xwalk

,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,a004x0000036LW4,a014x0000096Q2O,GWARL201905UIUX3,BC-UIUX
1,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,a004x0000036LVu,a014x0000096Q2E,GWDC201805DATA3,BC-DATAVIZ
3,a004x0000036LVz,a014x0000096Q2J,GWARL201905WEB3,BC-WEBDEV


In [85]:
# for rec in recs_to_delete:
#     try:
#         sf.Class_Participant__c.delete(rec['Name'])
#     except Exception as e:
#         print(e)

In [86]:
class_participant_load = pd.merge(class_participant_load, sf_student_xwalk)

class_participant_load = class_participant_load[['ID_Student__c','ID_Class__c','StartDate__c','EndDate__c']]

class_participant_load['StartDate__c'] = class_participant_load['StartDate__c'].astype(str)
class_participant_load['EndDate__c'] = class_participant_load['EndDate__c']

class_participant_load

class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records

[{'ID_Student__c': 'a024x000002cwzh',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwxH',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwxM',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwzm',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwzr',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwxR',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwy1',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Student__c': 'a024x000002cwzc',
  'ID_Class__c': 'a004x0000036LVp',
  'StartDate__c': '2020

In [89]:
for row in class_participant_records:
    # print(row)
    try:
        sf.ClassParticipant__c.create(row)
    except Exception as e:
        print(e)

In [90]:
# for rec in class_participant_records:

#     record = {
#         'ID_Class__c': rec['ID_Class__c'],
#         'ID_Student__c': rec['ID_Student__c'],
#         'StartDate__c': rec['StartDate__c'],
#         'EndDate__c': rec['EndDate__c'],
#     }
    
#     try:
#         sf.Classparticipant__c.create(record)
#     except Exception as e:
#         print(e)

# Load Staffassignment 

## Delete logic

In [91]:
staff_assn_records_del = sf.query("SELECT Name FROM Staffassignment__c")
recs_to_delete = [{'Name': r['Name']} for r in staff_assn_records_del['records']]
recs_to_delete

for rec in recs_to_delete:
    try:
        sf.Staffassignment__c.delete(rec['Name'])
    except Exception as e:
        print(e)



## Data Processing

In [95]:
sf_staff_records = []

# NameError: name 'sf_Staff_data' is not defined


sf_staff_data = sf.query('SELECT Name, Lastname__c, Middlename__c, Firstname__c, Birthdate__c, Employeeid__c FROM Staff__c')

for row in sf_staff_data['records']:
    
    rec = {
        'ID_Staff__c':row['Name'],
        'Lastname__c': row['Lastname__c'],
        'Middlename__c': row['Middlename__c'],
        'Firstname__c':row['Firstname__c'],
        'Birthdate__c': row['Birthdate__c'],
        'Employeeid__c': row['Employeeid__c']
        
    }
    
    sf_staff_records.append(rec)
 
sf_staff_records

sf_staff_xwalk = pd.DataFrame(sf_staff_records)
sf_staff_xwalk.head()

,ID_Staff__c,Lastname__c,Middlename__c,Firstname__c,Birthdate__c,Employeeid__c
0,a034x000001HF6T,Luongo,Nico,Darick,1995-07-05,000184220
1,a034x000001HF6Y,Sanford,Blair,Gemini,1992-04-22,000130109
2,a034x000001HF6d,Williams,De Angelo,Dartanion,1993-05-21,000160655
3,a034x000001HF6i,Rodney,Nicole,Heather,1994-06-07,000159108


In [96]:
query  ='''

select
    co.CourseCode,
    cl.Section,
    s.EmployeeID,
    sa.role, 
    sa.startdate,
    sa.enddate
from
    staffassignment as sa
    inner join staff s
    on sa.ID_Staff = s.ID_staff
    inner join class cl
    on sa.ID_Class=cl.ID_Class
    inner join course co
    on co.ID_Course=cl.ID_Course

'''
staffassignment_df = pd.read_sql(query, conn)
staffassignment_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None


In [97]:
#staffassignment_df
#sf_staff_xwalk
#sf_class_xwalk

In [98]:
print(staffassignment_df.columns)
print(sf_staff_xwalk.columns)
print(sf_class_xwalk.columns)



Index(['CourseCode', 'Section', 'EmployeeID', 'role', 'startdate', 'enddate'], dtype='object')
Index(['ID_Staff__c', 'Lastname__c', 'Middlename__c', 'Firstname__c',
       'Birthdate__c', 'Employeeid__c'],
      dtype='object')
Index(['ID_Class__c', 'ID_Course__c', 'Section__c', 'Course_Code__c'], dtype='object')


In [99]:
staff_assignment_load_df = pd.merge(staffassignment_df, sf_class_xwalk, left_on=['CourseCode','Section'], right_on=['Course_Code__c','Section__c'])
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [100]:
staff_assignment_load_df = pd.merge(staff_assignment_load_df, sf_staff_xwalk, left_on='EmployeeID', right_on='Employeeid__c')
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Class__c,ID_Course__c,Section__c,Course_Code__c,ID_Staff__c,Lastname__c,Middlename__c,Firstname__c,Birthdate__c,Employeeid__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000001HF6T,Luongo,Nico,Darick,1995-07-05,000184220
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000001HF6i,Rodney,Nicole,Heather,1994-06-07,000159108
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a004x0000036LVp,a014x0000096Q2E,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000001HF6d,Williams,De Angelo,Dartanion,1993-05-21,000160655


In [101]:
staff_assignment_load_df = staff_assignment_load_df[['ID_Staff__c','ID_Class__c','role','startdate','enddate']]
staff_assignment_load_df

,ID_Staff__c,ID_Class__c,role,startdate,enddate
0,a034x000001HF6T,a004x0000036LVp,Teacher Assistant,2020-09-16,None
1,a034x000001HF6i,a004x0000036LVp,Teacher Assistant,2020-09-16,None
2,a034x000001HF6d,a004x0000036LVp,Instructor,2020-09-16,None


In [102]:
staff_assignment_load_df.rename(columns={
    'role':'Role__c',
    'startdate':'Startdate__c',
    'enddate':'Enddate__c'
}, inplace=True)

staff_assignment_load_df['Startdate__c'] = staff_assignment_load_df['Startdate__c'].astype(str)

staff_assignment_load_df

,ID_Staff__c,ID_Class__c,Role__c,Startdate__c,Enddate__c
0,a034x000001HF6T,a004x0000036LVp,Teacher Assistant,2020-09-16,None
1,a034x000001HF6i,a004x0000036LVp,Teacher Assistant,2020-09-16,None
2,a034x000001HF6d,a004x0000036LVp,Instructor,2020-09-16,None


In [103]:
staff_assignment_records = staff_assignment_load_df.to_dict(orient='records')
staff_assignment_records

[{'ID_Staff__c': 'a034x000001HF6T',
  'ID_Class__c': 'a004x0000036LVp',
  'Role__c': 'Teacher Assistant',
  'Startdate__c': '2020-09-16',
  'Enddate__c': None},
 {'ID_Staff__c': 'a034x000001HF6i',
  'ID_Class__c': 'a004x0000036LVp',
  'Role__c': 'Teacher Assistant',
  'Startdate__c': '2020-09-16',
  'Enddate__c': None},
 {'ID_Staff__c': 'a034x000001HF6d',
  'ID_Class__c': 'a004x0000036LVp',
  'Role__c': 'Instructor',
  'Startdate__c': '2020-09-16',
  'Enddate__c': None}]

In [106]:
for rec in staff_assignment_records:
    try:
        sf.Staffassignment__c.create(rec)
    except Exception as e:
        print(e)

In [ ]:
#THE END~